In [ ]:
#From https://github.com/emanuel-metzenthin/Lime-For-Time/blob/master/demo/LIME-Pipeline.ipynb
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score as acc
import numpy as np
from lime import explanation
from lime import lime_base
import math


In [ ]:
!wget https://raw.githubusercontent.com/emanuel-metzenthin/Lime-For-Time/master/demo/data/coffee_train.csv
!wget https://raw.githubusercontent.com/emanuel-metzenthin/Lime-For-Time/master/demo/data/coffee_test.csv


In [ ]:
coffee_train = pd.read_csv('coffee_train.csv', sep=',', header=None).astype(float)
coffee_train_y = coffee_train.loc[:, 0]
coffee_train_x = coffee_train.loc[:, 1:]
coffee_test = pd.read_csv('coffee_test.csv', sep=',', header=None).astype(float)
coffee_test_y = coffee_test.loc[:, 0]
coffee_test_x = coffee_test.loc[:, 1:]


In [ ]:
for idx in range(0, len(coffee_train)):
    if coffee_train_y[idx] == 0:
        plt.plot(coffee_train_x.loc[idx, :], 'b')
    else:
        plt.plot(coffee_train_x.loc[idx, :], 'r')
plt.title('Coffee Dataset')
plt.show()


In [ ]:
knn = KNN()
knn.fit(coffee_train_x, coffee_train_y)
knn2 = KNN()
# knn2.fit(ecg_train_x, ecg_train_y)
print('Accuracy KNN for coffee dataset: %f' % (acc(coffee_test_y, knn.predict(coffee_test_x))))
# print('Accuracy KNN for earthquake dataset: %f' % (acc(ecg_test_y, knn2.predict(ecg_test_x))))


In [ ]:
# from https://github.com/florian-huber/weather_prediction_dataset/blob/main/notebooks/machine_learning_classification_bbq.ipynb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('scale', MinMaxScaler()),
    ('model', RandomForestClassifier(n_estimators=500, max_depth=6, random_state=0))
])


pipe.fit(coffee_train_x, coffee_train_y)

print('Accuracy pipe for coffee dataset: %f' % (acc(coffee_test_y, pipe.predict(coffee_test_x))))


In [ ]:
# From http://onnx.ai/sklearn-onnx/
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(pipe, initial_types=initial_type)
with open("coffee.onnx", "wb") as f:
    f.write(onx.SerializeToString())